In [1]:
import urllib
from bs4 import BeautifulSoup as bs
import pandas
import numpy as np
import os
from datetime import timedelta, date
from pprint import pprint
import re
import time
pandas.set_option('max_colwidth', -1)
#import dateutil

g = 0

def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)


print("Done")

Done


In [2]:
s = time.time()
alldates = []

namestartfinder = re.compile('^\d+[\-|\s|\–|(]+')
deadfinder = re.compile('\(d\.\s+\d+\)')
referencefinder = re.compile('[\,\s]+\[+\d*\]+')

all_births_df = pandas.DataFrame(data=None, columns=['themonth', 'theday', 'thetext'])
all_deaths_df = pandas.DataFrame(data=None, columns=['themonth', 'theday', 'thetext'])

start_dt = date(2020, 1, 1)  # pick 2020 so we get February 29
end_dt = date(2020, 12, 31)
special_cases = []
bcnt = 0
dcnt = 0

for dt in daterange(start_dt, end_dt):
    alldates.append(dt)    

for i in range(0, len(alldates)):
    today_births_df = pandas.DataFrame(data=None, columns=['thetext'])
    today_deaths_df = pandas.DataFrame(data=None, columns=['thetext'])
    thisurl = '\thttps://en.wikipedia.org/wiki/{0:}'.format(alldates[i].strftime('%B_%e').replace(" ",""))
    if ( (np.mod(i,20) == 0) | (i == len(alldates)-1) ):
        print('Parsing {0:}...'.format(alldates[i].strftime('%B %e')))
    #print(thisurl)
    page = urllib.request.urlopen(thisurl).read()
    soup = bs(page, "html.parser")
    sections = soup.findAll('h2')
    for this_section in sections:
        if (this_section.find('span', {'class': 'mw-headline', 'id': 'Births'}) != None):
            for this_birth in this_section.find_next_sibling('ul').findAll('li'):
                today_births_df.loc[bcnt, 'thetext'] = this_birth.get_text()
                bcnt += 1

        if (this_section.find('span', {'class': 'mw-headline', 'id': 'Deaths'}) != None):
            for this_birth in this_section.find_next_sibling('ul').findAll('li'):
                today_deaths_df.loc[dcnt, 'thetext'] = this_birth.get_text()
                dcnt += 1

    today_births_df = today_births_df.assign(themonth = np.nan)
    today_births_df = today_births_df.assign(theday = np.nan)
    today_deaths_df = today_deaths_df.assign(themonth = np.nan)
    today_deaths_df = today_deaths_df.assign(theday = np.nan)                
    
    today_births_df.loc[:, 'themonth'] = alldates[i].month
    today_births_df.loc[:, 'theday'] = alldates[i].day
    
    today_deaths_df.loc[:, 'themonth'] = alldates[i].month
    today_deaths_df.loc[:, 'theday'] = alldates[i].day
        
    all_births_df = pandas.concat((all_births_df, today_births_df), axis=0, sort=False)
    all_deaths_df = pandas.concat((all_deaths_df, today_deaths_df), axis=0, sort=False)


all_births_df.loc[:, 'themonth'] = pandas.to_numeric(all_births_df['themonth'], downcast='integer', errors='coerce')
all_births_df.loc[:, 'theday'] = pandas.to_numeric(all_births_df['theday'], downcast='integer', errors='coerce')
all_deaths_df.loc[:, 'themonth'] = pandas.to_numeric(all_deaths_df['themonth'], downcast='integer', errors='coerce')
all_deaths_df.loc[:, 'theday'] = pandas.to_numeric(all_deaths_df['theday'], downcast='integer', errors='coerce')

all_births_df = all_births_df.reset_index(drop=True)
all_deaths_df = all_deaths_df.reset_index(drop=True)

all_births_df.index.name = 'arbitraryIndex'
all_deaths_df.index.name = 'arbitraryIndex'

# print('saving for transfer to Kaggle...')
all_births_df.to_csv('births_wikipedia.csv', encoding='utf-8')
all_deaths_df.to_csv('deaths_wikipedia.csv', encoding='utf-8')

print('backing up...')
all_births_df_bk = all_births_df
all_deaths_df_bk = all_deaths_df
all_births_df_justparsed = all_births_df
all_deaths_df_justparsed = all_deaths_df


e = time.time()
g = g + (e-s)
print('Found {0:,.0f} births and {1:,.0f} deaths in {2:,.0f} minutes {3:,.0f} seconds!'.format(len(all_births_df), len(all_deaths_df), np.floor((e-s)/60), np.floor((e-s)%60)))




Parsing January  1...
Parsing January 21...
Parsing February 10...
Parsing March  1...
Parsing March 21...
Parsing April 10...
Parsing April 30...
Parsing May 20...
Parsing June  9...
Parsing June 29...
Parsing July 19...
Parsing August  8...
Parsing August 28...
Parsing September 17...
Parsing October  7...
Parsing October 27...
Parsing November 16...
Parsing December  6...
Parsing December 26...
Parsing December 31...
backing up...
Found 77,779 births and 35,838 deaths in 6 minutes 52 seconds!


In [7]:
s = time.time()
print('getting from backup...')
all_births_df = all_births_df_justparsed
all_deaths_df = all_deaths_df_justparsed

print('parsing birth years...')
print('\tassigning year...')
all_births_df = all_births_df.assign(birth_year = np.nan)
all_births_df.loc[all_births_df['thetext'].apply(lambda x: re.search('^\d+', x) != None), 'birth_year'] = pandas.to_numeric(all_births_df[all_births_df['thetext'].apply(lambda x: re.search('^\d+', x) != None)]['thetext'].apply(lambda x: x[re.search('^\d+', x).start():re.search('^\d+', x).end()]), errors='coerce')
print('\tfixing AD years...')
all_births_df.loc[(all_births_df['birth_year'].isnull()) & (all_births_df['thetext'].apply(lambda x: re.search('^AD\s*\d+', x))), 'birth_year'] = pandas.to_numeric(all_births_df[(all_births_df['birth_year'].isnull()) & (all_births_df['thetext'].apply(lambda x: re.search('^AD\s*\d+', x)))]['thetext'].apply(lambda x: x[re.search('^AD\s', x).end():re.search('^AD\s*\d+', x).end()]), errors='coerce')
print('\tsome special cases...')
all_births_df.loc[(all_births_df['birth_year'].isnull()) & (all_births_df['thetext'].apply(lambda x: x.strip() == 'Before 160 – Julia Maesa, Roman noblewoman (d. 224)[12]')), 'birth_year'] = 160
all_births_df.loc[(all_births_df['birth_year'].isnull()) & (all_births_df['thetext'].apply(lambda x: x.strip() == '774 – Kūkai, Japanese Buddhist monk, founder of Esoteric (Shingon) Buddhism (d. 835)[4]')), 'birth_year'] = 774
all_births_df.loc[(all_births_df['birth_year'].isnull()) & (all_births_df['thetext'].apply(lambda x: x.strip() == '[[1977   – Zulfiqer Russell, Bangladeshi journalist and lyricist')), 'birth_year'] = 1977
print('\tintegerizing...')
all_births_df.loc[:, 'birth_year'] = pandas.to_numeric(all_births_df['birth_year'], downcast='integer', errors='coerce')

print('parsing death years...')
print('\tassigning year...')
all_deaths_df = all_deaths_df.assign(death_year = np.nan)
all_deaths_df.loc[all_deaths_df['thetext'].apply(lambda x: re.search('^\d+', x) != None), 'death_year'] = pandas.to_numeric(all_deaths_df[all_deaths_df['thetext'].apply(lambda x: re.search('^\d+', x) != None)]['thetext'].apply(lambda x: x[re.search('^\d+', x).start():re.search('^\d+', x).end()]), errors='coerce')
print('\tfixing AD years...')
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['thetext'].apply(lambda x: re.search('^AD\s*\d+', x))), 'death_year'] = pandas.to_numeric(all_deaths_df[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['thetext'].apply(lambda x: re.search('^AD\s*\d+', x)))]['thetext'].apply(lambda x: x[re.search('^AD\s', x).end():re.search('^AD\s*\d+', x).end()]), errors='coerce')
print('\tfixing people who died on February 29th...')
all_deaths_df.loc[(all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Pat Garrett, American sheriff (b. 1850)')) | (all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'John Hope, 1st Marquess of Linlithgow, Scottish-Australian politician, 1st  Governor-General of Australia (b. 1860)')), 'death_year'] = 1908
all_deaths_df.loc[(all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Adolphe Appia, Swiss architect and theorist (b. 1862)')) | (all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Ina Coolbrith, American poet and librarian (b. 1841)')), 'death_year'] = 1928
all_deaths_df.loc[(all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Lena Blackburne, American baseball player, coach and manager (b. 1886)')) | (all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Tore Ørjasæter, Norwegian poet and educator (b. 1886)')), 'death_year'] = 1968
all_deaths_df.loc[(all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Yigal Allon, Israeli general and politician,  Prime Minister of Israel (b. 1918)')) | (all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Gil Elvgren, American painter and illustrator (b. 1914)')), 'death_year'] = 1980
all_deaths_df.loc[(all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Wes Farrell, American singer-songwriter and producer (b. 1939)')) | (all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Ralph Rowe, American baseball player, coach and manager (b. 1924)')), 'death_year'] = 1996
all_deaths_df.loc[(all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Janet Kagan, American author (b. 1946)')) | (all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Erik Ortvad, Danish painter and illustrator (b. 1917)'))| (all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Akira Yamada, Japanese scholar and philosopher (b. 1922)')), 'death_year'] = 2008
all_deaths_df.loc[(all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Roland Bautista, American guitarist (b. 1951)')) | (all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Davy Jones, English singer, guitarist and actor (b. 1945)'))| (all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Sheldon Moldoff, American illustrator (b. 1920)'))| (all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'P. K. Narayana Panicker, Indian social leader (b. 1930)')), 'death_year'] = 2012
all_deaths_df.loc[(all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Wenn V. Deramas, Filipino director and screenwriter (b. 1966)')) | (all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Gil Hill, American police officer, actor and politician (b. 1931)'))| (all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Josefin Nilsson, Swedish singer (b. 1969)'))| (all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Louise Rennison, English author (b. 1951)[12]'))| (all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Mumtaz Qadri, Pakistani assassin (b. 1985)[13]')), 'death_year'] = 2016
all_deaths_df.loc[(all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Robert Barrington-Ward, English lawyer and journalist (b. 1891)')) | (all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Rebel Oakes, American baseball player and manager (b. 1883)' )), 'death_year'] = 1948
all_deaths_df.loc[(all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Melvin Purvis, American police officer and  FBI agent (b. 1903)')) | (all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Walter Yust, American journalist and author (b. 1894)')), 'death_year'] = 1960
all_deaths_df.loc[(all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Kagamisato Kiyoji, Japanese sumo wrestler, the 42nd  Yokozuna (b. 1923)')) | (all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Jerome Lawrence, American playwright and author (b. 1915)'))| (all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Harold Bernard St. John, Barbadian lawyer and politician, 3rd  Prime Minister of Barbados (b. 1931)'))| (all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Lorrie Wilmot, South African cricketer (b. 1943)')), 'death_year'] = 2004
all_deaths_df.loc[(all_deaths_df['thetext'].apply(lambda x: str(x).strip() == 'Benjamin Sheares, Singaporean professor and politician, 2nd President of Singapore (b. 1907)[57]')), 'death_year'] = 1981

print('\tfixing people who died together in single events...')
all_deaths_df = all_deaths_df[~(all_deaths_df['thetext'].apply(lambda x: x[0:10] == 'Casualties'))].reset_index(drop=True)

all_deaths_df.loc[all_deaths_df['thetext'].apply(lambda x: str(x).strip() in ['Roger B. Chaffee, American pilot, engineer, and astronaut (b. 1935)', 'Gus Grissom, American pilot and astronaut (b. 1926)', 'Ed White, American colonel, engineer, and astronaut (b. 1930)']), 'death_year'] = 1967
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 1) & (all_deaths_df['theday'] == 28), 'death_year'] = 1986
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 2) & (all_deaths_df['theday'] == 1), 'death_year'] = 2003
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 2) & (all_deaths_df['theday'] == 3), 'death_year'] = 1959
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 2) & (all_deaths_df['theday'] == 6), 'death_year'] = 1958
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 2) & (all_deaths_df['theday'] == 12), 'death_year'] = 2009
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 2) & (all_deaths_df['theday'] == 26), 'death_year'] = 1936
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 3) & (all_deaths_df['theday'] == 6), 'death_year'] = 1836
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 3) & (all_deaths_df['theday'] == 24), 'death_year'] = 2015
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 4) & (all_deaths_df['theday'] == 10), 'death_year'] = 2010
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 4) & (all_deaths_df['theday'] == 15), 'death_year'] = 1912
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 4) & (all_deaths_df['theday'] == 20), 'death_year'] = 1999
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 5) & (all_deaths_df['theday'] == 18), 'death_year'] = 1980
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 6) & (all_deaths_df['theday'] == 1), 'death_year'] = 2001
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 7) & (all_deaths_df['theday'] == 17) & ((all_deaths_df['thetext'].apply(lambda x: 'Russia' in x)) | (all_deaths_df['thetext'].apply(lambda x: 'Demidova' in x)) | (all_deaths_df['thetext'].apply(lambda x: 'Kharitonov' in x)) | (all_deaths_df['thetext'].apply(lambda x: 'Trupp' in x)) | (all_deaths_df['thetext'].apply(lambda x: 'Botkin' in x))), 'death_year'] = 1918
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 7) & (all_deaths_df['theday'] == 17) & ( (all_deaths_df['thetext'].apply(lambda x: 'Breistroff' in x)) | (all_deaths_df['thetext'].apply(lambda x: 'Dadi' in x)) | (all_deaths_df['thetext'].apply(lambda x: 'Hogan' in x)) | (all_deaths_df['thetext'].apply(lambda x: 'Johnson' in x))), 'death_year'] = 1996
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 7) & (all_deaths_df['theday'] == 17) & ( (all_deaths_df['thetext'].apply(lambda x: 'Davison' in x)) | (all_deaths_df['thetext'].apply(lambda x: 'Jay' in x)) | (all_deaths_df['thetext'].apply(lambda x: 'Lange' in x)) | (all_deaths_df['thetext'].apply(lambda x: 'Witteveen' in x))), 'death_year'] = 2014
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 9) & (all_deaths_df['theday'] == 6), 'death_year'] = 1972
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 9) & (all_deaths_df['theday'] == 7), 'death_year'] = 2011
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 9) & (all_deaths_df['theday'] == 11), 'death_year'] = 2001
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 10) & (all_deaths_df['theday'] == 14), 'death_year'] = 1943
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 10) & (all_deaths_df['theday'] == 16), 'death_year'] = 1946
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 11) & (all_deaths_df['theday'] == 23), 'death_year'] = 1974
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['themonth'] == 12) & (all_deaths_df['theday'] == 7), 'death_year'] = 1941
print('\ta few special cases...')
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['thetext'].apply(lambda x: x.strip() == '1907 – Giosuè Carducci, Italian poet and educator, Nobel Prize laureate (b. 1835)')), 'death_year'] = 1907
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['thetext'].apply(lambda x: x.strip() == '2007 – Nancy Adams, New Zealand botanist and illustrator (b. 1926)')), 'death_year'] = 2007
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['thetext'].apply(lambda x: x.strip() == '1703 – Archibald Campbell, 1st Duke of Argyll, Scottish general (b. 1658)')), 'death_year'] = 1703
all_deaths_df.loc[(all_deaths_df['death_year'].isnull()) & (all_deaths_df['thetext'].apply(lambda x: x.strip() == '2014   – Vincent L. McKusick, American lawyer and judge (b. 1921)')), 'death_year'] = 2014

print('\tintegerizing...')
all_deaths_df.loc[:, 'death_year'] = pandas.to_numeric(all_deaths_df['death_year'], downcast='integer', errors='coerce')

print('removing births before 1800 and deaths before 1900...')
all_births_df = all_births_df[all_births_df['birth_year'] >= 1800]
all_deaths_df = all_deaths_df[all_deaths_df['death_year'] >= 1900]


print('getting dates...')
all_births_df = all_births_df.assign(birthdate = all_births_df.apply(lambda row: '{0:04d}-{1:02d}-{2:02d}'.format(row['birth_year'], row['themonth'], row['theday']), axis=1))
all_deaths_df = all_deaths_df.assign(deathdate = all_deaths_df.apply(lambda row: '{0:04d}-{1:02d}-{2:02d}'.format(row['death_year'], row['themonth'], row['theday']), axis=1))

print('backing up...')
all_births_df_bk = all_births_df
all_deaths_df_bk = all_deaths_df

e = time.time()
g = g + (e-s)
print('Got {0:,.0f} births and {1:,.0f} deaths in {2:,.0f} seconds!'.format(len(all_births_df), len(all_deaths_df), e-s))
#all_deaths_df.shape
#all_deaths_df[all_deaths_df['theyear'].isnull()]['thetext'].apply(lambda x: "'"+x.strip()+"'")

#all_births_df[all_births_df['thetext'].apply(lambda x: 'laura bassi' in x.lower())]



getting from backup...
parsing birth years...
	assigning year...
	fixing AD years...
	some special cases...
	integerizing...
parsing death years...
	assigning year...
	fixing AD years...
	fixing people who died on February 29th...
	fixing people who died together in single events...
	a few special cases...
	integerizing...
removing births before 1800 and deaths before 1900...
getting dates...
backing up...
Got 71,368 births and 25,359 deaths in 13 seconds!


In [6]:
s = time.time()
print('Getting from backup...')
all_births_df = all_births_df_bk
all_deaths_df = all_deaths_df_bk

namefinder = re.compile('^\d+\s*[–|\-]')
deathfinder = re.compile('\(d\.\s*\d+\)\s*')
birthfinder = re.compile('\(b\.\s*\d+\)\s*')

print('Births...')
print('\tparsing text string...')
all_births_df = all_births_df.assign(name_and_more = np.nan)
all_births_df.loc[
    (all_births_df['thetext'].apply(lambda x: re.search(namefinder, x) != None)), 'name_and_more'
] = all_births_df[(all_births_df['thetext'].apply(lambda x: re.search(namefinder, x) != None))]['thetext'].apply(lambda x: x[re.search(namefinder,x).end():].strip())

print('\tgetting death year as a check for matching...')
all_births_df = all_births_df.assign(death_string = np.nan)
all_births_df.loc[all_births_df['thetext'].apply(lambda x: re.search(deathfinder, str(x)) != None), 
                  'death_string'] = all_births_df[all_births_df['thetext'].apply(lambda x: re.search(deathfinder, str(x)) != None)]['thetext'].apply(lambda x: x[re.search(deathfinder, str(x)).start():re.search(deathfinder, str(x)).end()])

all_births_df = all_births_df.assign(death_year_check = pandas.to_numeric(-1, downcast='integer', errors='coerce'))
all_births_df.loc[all_births_df['death_string'].notnull(), 'death_year_check'] = pandas.to_numeric(
    all_births_df[all_births_df['death_string'].notnull()]['death_string'].apply(lambda x: x[re.search('\d+',x).start():re.search('\d+',x).end()]), 
    downcast='integer', errors='coerce')

print('\tgetting names...')
all_births_df = all_births_df.assign(death_excised = np.nan)
all_births_df.loc[
    all_births_df['name_and_more'].apply(lambda x: re.search(deathfinder, str(x)) != None), 
    'death_excised'] = all_births_df[
                (all_births_df['name_and_more'].apply(lambda x: re.search(deathfinder, str(x)) != None))
                        ]['name_and_more'].apply(lambda x: x[:re.search(deathfinder, str(x)).start()] + x[re.search(deathfinder, str(x)).end():])
all_births_df.loc[all_births_df['death_excised'].isnull(), 'death_excised'] = all_births_df['name_and_more']
all_births_df = all_births_df.assign(thename = all_births_df['death_excised'].apply(lambda x: str(x).split(',')[0].strip()))

print('\tfixing a few special cases...')
all_births_df.loc[all_births_df['thetext'] == '1920(probable) – Isaac Asimov, American writer and professor of biochemistry (d. 1992)[84]', 'thename'] = 'Isaac Asimov'
all_births_df.loc[all_births_df['thetext'] == '1999 — Fernando Tatís Jr., American baseball player[144]', 'thename'] = 'Fernando Tatís Jr.'
all_births_df.loc[all_births_df['thetext'] == '1969 or 1970 – Mariah Carey, American singer-songwriter, producer, and actress[4]', 'thename'] = 'Mariah Carey'
all_births_df.loc[all_births_df['thetext'] == '1882 (O.S. 27 March) – Dmytro Doroshenko, Lithuanian-Ukrainian historian and politician, Minister of Foreign Affairs of Ukraine and Prime Minister of Ukraine (d. 1951)', 'thename'] = 'Dmytro Doroshenko'
all_births_df.loc[all_births_df['thetext'] == '[[1977   – Zulfiqer Russell, Bangladeshi journalist and lyricist', 'thename'] = 'Zulfiqer Russell'
print('\tremoving duplicates...')
all_births_df = all_births_df[~((all_births_df['thename'] == 'Wladimir Burliuk') & (all_births_df['themonth'] == 3) & (all_births_df['theday'] == 15))] # Drop old Russian system birthdate
all_births_df = all_births_df[~(all_births_df['thetext'] == '1951   – Theresa Hak Kyung Cha, South Korean-American author, director, and producer (d. 1982)')]
all_births_df = all_births_df[~((all_births_df['thename'] == 'Symon Petliura') & (all_births_df['themonth'] == 5) & (all_births_df['theday'] == 10))] # Drop old Russian system birthdate
all_births_df = all_births_df[~((all_births_df['thename'] == 'Manuel González Flores') & (all_births_df['themonth'] == 6) & (all_births_df['theday'] == 17) )]
all_births_df = all_births_df[~((all_births_df['thename'] == 'Maryam Mirzakhani') & (all_births_df['themonth'] == 5) & (all_births_df['theday'] == 3) )]
all_births_df = all_births_df[~((all_births_df['thename'] == 'Sarah Bernhardt') & (all_births_df['themonth'] == 10) & (all_births_df['theday'] == 23) )]
all_births_df = all_births_df[~((all_births_df['thename'] == 'Nâzım Hikmet') & (all_births_df['themonth'] == 11) & (all_births_df['theday'] == 20) )]
all_births_df = all_births_df[~((all_births_df['thename'] == 'Syreeta Wright') & (all_births_df['themonth'] == 2) & (all_births_df['theday'] == 28) )]
all_births_df = all_births_df[~((all_births_df['thename'] == 'Helenio Herrera') & (all_births_df['themonth'] == 4) & (all_births_df['theday'] == 17) )]
all_births_df = all_births_df[~((all_births_df['thename'] == 'Amalia Mendoza') & (all_births_df['themonth'] == 7) & (all_births_df['theday'] == 23) )]
all_births_df = all_births_df[~((all_births_df['thename'] == 'Valeriy Brumel') & (all_births_df['themonth'] == 5) & (all_births_df['theday'] == 14) )]
all_births_df = all_births_df[~((all_births_df['thename'] == 'Velimir Khlebnikov') & (all_births_df['themonth'] == 10) & (all_births_df['theday'] == 28) )]
all_births_df = all_births_df[~((all_births_df['thename'] == 'Bernhard Gregory') & (all_births_df['themonth'] == 4) & (all_births_df['theday'] == 10) )]
all_births_df = all_births_df[~((all_births_df['thename'] == 'Dimitrie Cuclin') & (all_births_df['themonth'] == 3) & (all_births_df['theday'] == 24) )]
print('\twhen we encounter two people with the same name and death year, modify the less famous one...')
all_births_df.loc[all_births_df['thetext'] == '1925 – Ted Kennedy, Canadian ice hockey player (d. 2009)[50]', 'thename'] = 'Ted Kennedy (Canadian ice hockey player)'
all_births_df.loc[all_births_df['thetext'] == '1941   – Charles Keating, English-American actor (d. 2014)', 'thename'] = 'Charles Keating (actor)'
all_births_df.loc[all_births_df['thetext'] == '1922 – Joe Frazier, American baseball player and manager (d. 2011)', 'thename'] = 'Joe Frazier (baseball player)'
all_births_df.loc[all_births_df['thetext'] == '1910 – Mary Jackson, American actress (d. 2005)', 'thename'] = 'Mary Jackson (actress)'
all_births_df.loc[all_births_df['thetext'] == '1924 – Ingemar Johansson, Swedish race walker (d. 2009)', 'thename'] = 'Ingemar Johansson (race walker)'
all_births_df.loc[all_births_df['thetext'] == '1931   – Dickie Moore, Canadian ice hockey player and businessman (d. 2015)', 'thename'] = 'Dickie Moore (Canadian ice hockey player)'
all_births_df.loc[all_births_df['thetext'] == '1923   – Peter Underwood, English parapsychologist and author (d. 2014)', 'thename'] = 'Peter Underwood (parapsychologist)'
    
print('Deaths...')
print('\tparsing text string...')
all_deaths_df = all_deaths_df.assign(name_and_more = np.nan)
all_deaths_df.loc[
    (all_deaths_df['thetext'].apply(lambda x: re.search(namefinder, x) != None)), 'name_and_more'
] = all_deaths_df[(all_deaths_df['thetext'].apply(lambda x: re.search(namefinder, x) != None))]['thetext'].apply(lambda x: x[re.search(namefinder,x).end():].strip())

print('\tgetting birth year as a check for matching...')
all_deaths_df = all_deaths_df.assign(birth_string = np.nan)
all_deaths_df.loc[all_deaths_df['thetext'].apply(lambda x: re.search(birthfinder, str(x)) != None), 
                  'birth_string'] = all_deaths_df[all_deaths_df['thetext'].apply(lambda x: re.search(birthfinder, str(x)) != None)]['thetext'].apply(lambda x: x[re.search(birthfinder, str(x)).start():re.search(birthfinder, str(x)).end()])

all_deaths_df = all_deaths_df.assign(birth_year_check = pandas.to_numeric(-1, downcast='integer', errors='coerce'))
all_deaths_df.loc[all_deaths_df['birth_string'].notnull(), 'birth_year_check'] = pandas.to_numeric(
    all_deaths_df[all_deaths_df['birth_string'].notnull()]['birth_string'].apply(lambda x: x[re.search('\d+',x).start():re.search('\d+',x).end()]), 
    downcast='integer', errors='coerce')

print('\tgetting names...')
all_deaths_df = all_deaths_df.assign(birth_excised = np.nan)
all_deaths_df.loc[
    all_deaths_df['name_and_more'].apply(lambda x: re.search(deathfinder, str(x)) != None), 
    'birth_excised'] = all_deaths_df[
                (all_deaths_df['name_and_more'].apply(lambda x: re.search(deathfinder, str(x)) != None))
                        ]['name_and_more'].apply(lambda x: x[:re.search(deathfinder, str(x)).start()] + x[re.search(deathfinder, str(x)).end():])
all_deaths_df.loc[all_deaths_df['birth_excised'].isnull(), 'birth_excised'] = all_deaths_df['name_and_more']
all_deaths_df = all_deaths_df.assign(thename = all_deaths_df['birth_excised'].apply(lambda x: str(x).split(',')[0].strip()))



print('\tFixing weirdness with names for people who died together...')
all_deaths_df.loc[all_deaths_df['thename'] == 'nan', 'thename'] = all_deaths_df[all_deaths_df['thename'] == 'nan']['thetext'].apply(lambda x: re.split(',',x)[0])
all_deaths_df = all_deaths_df[~(all_deaths_df['thename'].apply(lambda x: re.search('crew', x) != None))]
all_deaths_df = all_deaths_df[~(all_deaths_df['thetext'].apply(lambda x: re.search('Casualties', x) != None))]
all_deaths_df.loc[(all_deaths_df['thetext'].apply(lambda x: re.search('^\s*(–|-)\s*', x) != None)), 'thename'] = all_deaths_df[(all_deaths_df['thetext'].apply(lambda x: re.search('^\s*(–|-)\s*', x) != None))][ 'thename'].apply(lambda x: x[re.search('^\s*(–|-)\s*', str(x)).end():])
all_deaths_df = all_deaths_df.loc[~(all_deaths_df['thetext'].apply(lambda x: 'attack' in x.lower()))]                              
all_deaths_df = all_deaths_df[~all_deaths_df['thetext'].apply(lambda x: 'incident' in x.lower())]

print('\tRemoving duplicates...')
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'David Allen') & (all_deaths_df['themonth'] == 5) & (all_deaths_df['theday'] == 25))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Francisco Franco') & (all_deaths_df['themonth'] == 11) & (all_deaths_df['theday'] == 19))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Dirk Jan de Geer') & (all_deaths_df['themonth'] == 11) & (all_deaths_df['theday'] == 27))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Jyoti Basu') & (all_deaths_df['themonth'] == 1) & (all_deaths_df['theday'] == 16))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'John Marin') & (all_deaths_df['themonth'] == 10) & (all_deaths_df['theday'] == 1))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Bret Harte') & (all_deaths_df['themonth'] == 5) & (all_deaths_df['theday'] == 6))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'George Bouzianis') & (all_deaths_df['themonth'] == 10) & (all_deaths_df['theday'] == 22))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Udo Lattek') & (all_deaths_df['themonth'] == 2) & (all_deaths_df['theday'] == 1))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Jun Seba') & (all_deaths_df['themonth'] == 2) & (all_deaths_df['theday'] == 10))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'George Thorn') & (all_deaths_df['themonth'] == 1) & (all_deaths_df['theday'] == 13))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'George Hees') & (all_deaths_df['themonth'] == 6) & (all_deaths_df['theday'] == 10))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Edward Greenspan') & (all_deaths_df['themonth'] == 12) & (all_deaths_df['theday'] == 23))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Abdul Haris Nasution') & (all_deaths_df['themonth'] == 9) & (all_deaths_df['theday'] == 5))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Alfred de Grazia') & (all_deaths_df['themonth'] == 7) & (all_deaths_df['theday'] == 12))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Edmonia Lewis') & (all_deaths_df['death_year'] == 1911) )]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Laura Ingalls Wilder') & (all_deaths_df['themonth'] == 1) & (all_deaths_df['theday'] == 10))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Jean Moréas') & (all_deaths_df['themonth'] == 4) & (all_deaths_df['theday'] == 30))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Max Linder') & (all_deaths_df['themonth'] == 10) & (all_deaths_df['theday'] == 31))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Flora MacNeil') & (all_deaths_df['themonth'] == 5) & (all_deaths_df['theday'] == 16))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Tony Hancock') & (all_deaths_df['themonth'] == 6) & (all_deaths_df['theday'] == 24))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Charlie Byrd') & (all_deaths_df['themonth'] == 11) & (all_deaths_df['theday'] == 30))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Gene Raymond') & (all_deaths_df['themonth'] == 5) & (all_deaths_df['theday'] == 2))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Robert Mensah') & (all_deaths_df['themonth'] == 9) & (all_deaths_df['theday'] == 2))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'M. Visvesvaraya') & (all_deaths_df['themonth'] == 1) & (all_deaths_df['theday'] == 14))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Tito Puente') & (all_deaths_df['themonth'] == 5) & (all_deaths_df['theday'] == 31))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Bill Cowsill') & (all_deaths_df['themonth'] == 2) & (all_deaths_df['theday'] == 17))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Rebecca Clarke') & (all_deaths_df['death_year'] == 1981) )]

print('\tfixing people who died on February 29th...')
all_deaths_df = all_deaths_df[~all_deaths_df['thename'].apply(lambda x: re.search('^\d+\n+',x) != None)]#.shape#[['thename', 'death_text']]

print('\tfixing senior/junior...')
# all_deaths_df.loc[(all_deaths_df['thetext'] == 'Arthur M. Schlesinger') & (all_deaths_df['death_year'] == 1965), 'thename'] = 'Arthur M. Schlesinger, Sr.'
# all_deaths_df.loc[(all_deaths_df['thetext'] == 'Arthur M. Schlesinger') & (all_deaths_df['death_year'] == 2007), 'thename'] = 'Arthur M. Schlesinger, Jr.'
# all_deaths_df.loc[(all_deaths_df['thetext'] == 'Barack Obama') & (all_deaths_df['death_year'] == 1982), 'thename'] = 'Barack Obama, Sr.'
# all_deaths_df.loc[(all_deaths_df['thetext'] == 'John Paul Getty') & (all_deaths_df['death_year'] == 2003), 'thename'] = 'John Paul Getty, Jr.'
# all_deaths_df.loc[(all_deaths_df['thetext'] == 'Cal Ripken') & (all_deaths_df['death_year'] == 1999), 'thename'] = 'Cal Ripken, Sr.'




print('matching names...')
all_deaths_df = all_deaths_df[all_deaths_df['thetext'] != '1981']
all_births_df.loc[all_births_df['thetext'] == '1901 – Nina Bari, Russian mathematician (d. 1961)[11]', 'thename'] = 'Nina Bari'
all_deaths_df.loc[all_deaths_df['thetext'] == '1961.  – Nina Bari, Russian mathematician (b. 1901)[25]', 'thename'] = 'Nina Bari'
all_births_df.loc[all_births_df['thetext'] == '1915 – Eugène Martin, French racing driver (d. 2006)', 'thename'] = 'Eugène Martin'
all_deaths_df.loc[all_deaths_df['thetext'] == '2006.  – Eugène Martin, French race car driver (b. 1915)', 'thename'] = 'Eugène Martin'
all_births_df.loc[all_births_df['thetext'] == '1876 – 13th Dalai Lama (d. 1933)', 'thename'] = '13th Dalai Lama'




print('removing reference marks from names...')
all_births_df.loc[all_births_df['thename'].apply(lambda x: re.search('\[\d+\]',str(x)) != None)
                  , 'thename'] = all_births_df[
    all_births_df['thename'].apply(lambda x: re.search('\[\d+\]',str(x)) != None)
]['thename'].apply(lambda x: x[:re.search('\[\d+\]',str(x)).start()])

all_deaths_df.loc[
    all_deaths_df['thename'].apply(lambda x: re.search('\[\d+\]',str(x)) != None), 
    'thename'] = all_deaths_df[
    all_deaths_df['thename'].apply(lambda x: re.search('\[\d+\]',str(x)) != None)
]['thename'].apply(lambda x: x[:re.search('\[\d+\]',str(x)).start()])


print('renaming text columns...')
# all_births_df = all_births_df.rename(columns={'thetext': 'birth_text'})
# all_deaths_df = all_deaths_df.rename(columns={'thetext': 'death_text'})

print('backing up...')
# all_births_df_bk = all_births_df
# all_deaths_df_bk = all_deaths_df

e = time.time()
g = g + (e-s)
print('Found {0:,.0f} births and {1:,.0f} deaths in {2:,.0f} seconds!'.format(len(all_births_df), len(all_deaths_df), e-s))
#all_births_df[all_births_df['thetext'].apply(lambda x: 'laura bassi' in x.lower())]

#all_births_df[all_births_df['death_year_check'] != -1].groupby(['thename', 'death_year_check']).size().sort_values(ascending=False)



Getting from backup...
Births...
	parsing text string...
	getting death year as a check for matching...


KeyboardInterrupt: 

In [5]:
all_deaths_df[all_deaths_df['thetext'] == 'Arthur M. Schlesinger']

KeyError: 'thetext'

In [ ]:
print('\tfixing senior/junior...')
# all_deaths_df.loc[(all_deaths_df['thetext'] == 'Arthur M. Schlesinger') & (all_deaths_df['death_year'] == 1965), 'thename'] = 'Arthur M. Schlesinger, Sr.'
# all_deaths_df.loc[(all_deaths_df['thetext'] == 'Arthur M. Schlesinger') & (all_deaths_df['death_year'] == 2007), 'thename'] = 'Arthur M. Schlesinger, Jr.'
# all_deaths_df.loc[(all_deaths_df['thetext'] == 'Barack Obama') & (all_deaths_df['death_year'] == 1982), 'thename'] = 'Barack Obama, Sr.'
# all_deaths_df.loc[(all_deaths_df['thetext'] == 'John Paul Getty') & (all_deaths_df['death_year'] == 2003), 'thename'] = 'John Paul Getty, Jr.'
# all_deaths_df.loc[(all_deaths_df['thetext'] == 'Cal Ripken') & (all_deaths_df['death_year'] == 1999), 'thename'] = 'Cal Ripken, Sr.'


In [ ]:
df = all_births_df[['thename', 'birth_year', 'birthdate', 'death_year_check', 'birth_text']].merge(
    all_deaths_df[['thename', 'death_year', 'deathdate', 'birth_year_check', 'death_text']], 
    how='inner', left_on=['thename', 'birth_year'], right_on=['thename', 'birth_year_check']
).reset_index()



# all_births_df.loc[all_births_df['thetext'] == '1876 – 13th Dalai Lama (d. 1933)', 'thename'] = '13th Dalai Lama'
# all_deaths_df.loc[all_deaths_df['thetext'] == '1933 – 13th Dalai Lama (b. 1876)', 'thename'] = '13th Dalai Lama'
#
# all_deaths_df[
#     ~all_deaths_df['thename'].isin(df['thename'].drop_duplicates().tolist())
# ].sort_values(by='thename')


# 'Abdullah I', 'Abdul Haq', Abe Pollin', 'Adolphus Busch III', 'Adriaan Blaauw', 'Afeni Shakur', 
# 'Al Goldstein', 'Al Lucas', 'Alan Ball Jr.', 'Alan Hale', 'Alan Hale Jr.', 'Albert Dietrich',
# 'Albert Fish', 'Alec Holowka', 'Alex Pullin', ''Alexander Calder'', 'Alexander Onassis', 
# 'Alexander Litvinenko' ,'Alexander Volkov', 'Alexandre Yersin', 'Alferd Packer', 'Alfie Curtis', 
# 'Alfonso XIII of Spain (b. 1886)', 'Alfred A. Knopf Sr.', 'Allen Welsh Dulles', 'Amanda McKittrick Ros',
# 'André Kamperveen', 'Andrei Karlov', 'Angelo Errichetti', 'Anthony Burgess', 'Anwar al-Awlaki',
# 'Archibald Butt', 'Arnold Lobel', 'Art Donovan', 'Art Fleming', 'Arthur Chevrolet', 'Arthur Guinness', 
# 'Arthur M. Schlesinger, Sr.', 'Arthur M. Schlesinger, Jr.', 'Arthur Mitchell', 'Arthur Ochs Sulzberger'
# 'Arthur Seyss-Inquart', 'Augustus D. Juilliard', 'Baby Huey', 'Barnett Slepian', 'Barthélemy Boganda',
# 'Baudouin', 'Ben Bradlee', 'Benjamin Bloom', 'Bernard Lee', 'Betty Carter', 'Bill Bowerman', 'Bill Doak',
# 'Bill France Sr.', 'Billy Carter', 'Billy Mays', 'Bingham Ray', 'Birendra of Nepal (b. 1945)', 
# 'Blair Kiel', 'Bo Schembechler', 'Bob Jones Sr.', 'Bob Probert', 'Bobby Dodd', 'Bobby Riggs', 
# 'Bogd Khan', 'Bonnie Lee Bakley', 'Branch Rickey', 'Brandon Teena', 'Brendan Burke', 'Breonna Taylor',
# 'Brook Berringer', 'Bruce Edward Ivins', 'Bruce McCandless II', 'Bruce Snyder', 'Bud Abbott'
# 'Bud Adams', 'Buddy Cianci', 'Buenaventura Durruti', 'Byron Looper', 'Calamity Jane', 'Carl Weiss',
# 'Carlos I of Portugal', 'Carlos Prats', 'Carter Albrecht', 'Casey Calvert', 'Cassie Bernall',
# 'Charles Atlas', Charles Eames', 'Charles Hermite', 'Charles I', 'Charlotte', 'Che Guevara', 
# 'Chet Powers', 'Chief Joseph', 'Chris Kyle', 'Christina Onassis', 'Christine Chubbuck', 
# 'Christine Jorgensen', 'Chuck Bennett', 'Cixi', 'Claus von Bülow', 'Clement Studebaker',
# 'Constantine I of Greece (b. 1868)', 'Dan Blocker', 'Dan Rooney', 'Daniel M. Lewin', 'Danny Thomas',
# 'Darrell Royal', 'David Bloom', 'David Foster Wallace', 'David Koresh', 'David Mark Berger', 'David Rappaport',
# 'David T. Abercrombie', 'Dawn Brancheau', 'Dean Smith', 'Deng Liqun', 'Denver Pyle', 'Dermot Morgan', 
# 'Derrick Thomas', "Des O'Connor", 'Dhirendra of Nepal (b. 1950)', 'Dick Sargent', 'Dik Browne', 
# 'Dirk Jan de Geer', 'Don Haskins', 'Doris Grau', 'Dorothy Hale', 'Douglas Mawson', 'Earl Blaik', 
# 'Earl Woods', 'Eazy-E', 'Ed Delahanty', 'Ed Koch', 'Eddie Albert', 'Edsel Ford', 'Edvard Munch', 
# 'Edward VII of the United Kingdom (b. 1841)', 'Edward VIII of the United Kingdom (b. 1894)',
# 'Edwin G. Krebs', 'Elena Ceaușescu', 'Elias Disney', 'Elizabeth Taylor', 'Elliot See', 'Emily Hahn', 
# 'Emily Post', 'Emperor Meiji of Japan (b. 1852)', 'Emperor Taishō of Japan (b. 1879)', 'Enrico Caruso',
# 'Eric Andolsek', 'Ernest Gagnon', 'Ernest Graves', 'Ernie Ball', 'Ernst Happel', 'Etika', 'Etta Jones', 
# 'Eugie Foster', 'Euronymous', 'Eva Gabor', 'Fab Melo', 'Fabiola', ' Fahd of Saudi Arabia (b. 1923)', 
# 'Faisal I of Iraq (b. 1883)', 'Farouk of Egypt (b. 1920)', 'Father Divine', 'Ferdinand Alexander Porsche', 
# 'Flo Ziegfeld', 'Florence Bascom', 'Forrest Mars Sr.', 'Francis Hughes', 'Frank Gorshin', 'Frank Perdue', 
# 'Frank Sinatra Jr.', 'Frank Zamboni', 'Franz Ferdinand', 'Franz Joseph I of Austria (b. 1830)', 
# 'Franz Joseph II', 'François Michelin', 'Fred Steiner', 'Fredy Hirsch', 'Friedrich Nietzsche', 
# 'Fritz Pollard', 'Fuad I of Egypt (b. 1868)', 'Gaines Adams', 'Garo Yepremian', 'Garrick Utley',
# 'Gaston Chevrolet', 'Gene L. Coon', 'Gene Upshaw', 'Geoff Edwards', 'Geoffrey Beene', 'Georg von Trapp',
# 'George Cadbury', 'George Floyd', 'George Foster Peabody', 'George Gipp', 'George II', 'George Tiller', 
# 'George V of the United Kingdom (b. 1865)', 'George VI of the United Kingdom (b. 1895)', 
# 'George Washington Carver', 'Georgy Malenkov', 'Gerald Ford', 'Gertrude Bell', 'Gilles Tremblay', 
# 'Giuseppe Zangara', 'Goo Hara', 'Gord Downie', 'Gordon Jump', 'Gorgeous George', 
# 'Grand Duchess Anastasia Nikolaevna of Russia (b. 1901)', 'Grand Duchess Maria Nikolaevna of Russia (b. 1899)', 
# 'Grand Duchess Olga Alexandrovna of Russia (b. 1882)', 'Grand Duchess Olga Nikolaevna of Russia (b. 1895)', 
# 'Grand Duchess Tatiana Nikolaevna of Russia (b. 1897)', 'Grand Duke Peter Nikolaevich of Russia (b. 1864)', 
# 'Grigory Landsberg', 'Grover Washington Jr.', 'Gummo Marx', 'Gus Triandos', 'Gustaf V of Sweden (b. 1858)', 
# 'Gustaf VI Adolf of Sweden (b. 1882)', 'Gustav Klimt', 'Gérard Louis-Dreyfus', 'H. B. Reese', 
# 'H. John Heinz III', 'Haakon VII of Norway (b. 1872)', 'Hank Gathers', 'Harald Sverdrup', 'Harriet Tubman', 
# 'Harry Dean Stanton', 'Hector Boyardee', 'Hedy Lamarr', 'Hendrik Casimir', 'Henri Langlois', 
# 'Henrietta Lacks', 'Hiram Fong', 'Hiram Johnson', 'Hiroshi Yoshida', 'Howard Unruh', 'Hoyt Wilhelm', 
# 'Hua Guofeng', 'Hugh Trevor-Roper', 'Huo Yuanjia', 'Hussein bin Ali', 'Ibn Saud', 'Idi Amin', 'Igor Korolev', 
# 'Imre Nagy', 'Ingmar Bergman', 'Isaac C. Kidd', 'Isadora Duncan', 'Italo Calvino', 'Ivan Hlinka', 
# 'J. Bruce Ismay', 'J. G. Farrell', 'J. R. R. Tolkien', 'J.I. Rodale', 'Jack B. Sowards', 'Jack Chick', 
# 'Jack Daniel', 'Jack Lord', 'Jack Ruby', 'Jack Tramiel', 'Jacob Pressman', 'James Blish', 
# 'Jan Nieuwenhuys', 'Jan Sloot', 'Jane Fortune', 'Jane Roberts', 'Janet Munro', 'Jason Collier', 
# 'Jean Jaurès', 'Jeff Alm', 'Jelly Roll Morton', 'Jeremy Michael Boorda', 'Jesse White', 'Jessica Dubroff', 
# 'Jim Perry', 'Jim Stynes', 'Jim Thorpe', 'Jimmie Foxx', 'Joan Crawford', 'Joe Gordon', 'Joe Niekro', 
# 'Joe Tinker', 'Johann Breyer', 'John B. Watson', 'John Backus', 'John Bain (TotalBiscuit)', 'John Birch', 
# 'John Clayton Adams', 'John Colicos', 'John D. Dingell', 'John D. Rockefeller Jr.', 'John Daly', 
# 'John Foster Dulles', 'John Hay', 'John Howard Griffin', 'John Hoyt', 'John Joseph O'Connor', 
# 'John L. Sullivan', 'John Patrick Foley'


i = 4900
j = i
for ix, thisrow in all_deaths_df[
    ~all_deaths_df['thename'].isin(df['thename'].drop_duplicates().tolist())
].sort_values(by='thename')[i:i+100].iterrows():
    #print("{0:}...".format(thisrow['thename']))
    #print("j = {0:.0f}\t({1:.0f}):\t{2:}".format(j, ix, thisrow['thename']))#, thisrow['death_text']))   
    print("j = {0:.0f} ({1:.0f}): {2:}\n\t{3:}\n".format(j, ix, thisrow['thename'], thisrow['death_text']))
    j = j + 1
    
# all_deaths_df[
#     (~all_deaths_df['thename'].isin(df['thename'].drop_duplicates().tolist()))
# ][['thename', 'death_text']].sort_values(by='thename').to_csv('names_to_check.csv', encoding='utf-8')
# print('ok')
# 4999 (34864): John Robert Beyster


In [ ]:
all_deaths_df[all_deaths_df['death_text'] == 'John Paul Getty, Jr.']

In [ ]:
all_deaths_df.loc[(all_deaths_df['thetext'] == 'Barack Obama') & (all_deaths_df['death_year'] == 1982), 'thename'] = 'Barack Obama, Sr.'
all_deaths_df.loc[(all_deaths_df['thetext'] == 'John Paul Getty') & (all_deaths_df['death_year'] == 2003), 'thename'] = 'John Paul Getty, Jr.'


In [ ]:
s = time.time()
print('Getting from backup...')
df = df_bk

i = 0


#


#all_deaths_df[all_deaths_df['thename'].apply(lambda x: 'Gus Grissom' in x)]


In [ ]:

i = #1300
for ix, thisrow in all_deaths_df[
     ~(all_deaths_df['thename'].isin(df['thename'].drop_duplicates().tolist()))
][['thename', 'death_text']].sort_values(by='thename')[i:i+100].iterrows():
    print("{0}  '{1:}'\n\t{2:}\n".format(i, thisrow['thename'], thisrow['death_text']))
    i += 1
    #.apply(lambda row: "'{0:}',       {1:}'".format(row['thename'], row['death_text']), axis=1) #.sort_values(by='thename')[800:850]


In [ ]:


#all_births_df[all_births_df['birth_text'].apply(lambda x: 'hickok' in x.lower())]

In [ ]:


print('removing records with no birthdate AND no deathdate...')
df = df[~((df['birthdate'].isnull()) & (df['deathdate'].isnull()))]

print('{0:,.0f} records have both birth and death dates...'.format(len(df[((df['birthdate'].notnull()) & (df['deathdate'].notnull()))])))
print('{0:,.0f} records have birthdates but no deathdates...'.format(len(df[((df['birthdate'].notnull()) & (df['deathdate'].isnull()))])))
print('{0:,.0f} records have deathdates but no birthdates...'.format(len(df[((df['birthdate'].isnull()) & (df['deathdate'].notnull()))])))


print('removing records with no birthdate AND no deathdate...')
df = df[(df['birthdate'].notnull()) & (df['deathdate'].notnull())]

print('dateifying dates...')
df.loc[:, 'birthdate'] = pandas.to_datetime(df['birthdate'])
df.loc[:, 'deathdate'] = pandas.to_datetime(df['deathdate'])

print('calculating Cobain dates...')
df = df.assign(cobaindate = df.apply(lambda row: row['deathdate'] + (row['deathdate'] - row['birthdate']), axis=1))

print('backing up...')
df_bk = df

e = time.time()
g = g + (e-s)
print('\nFound Cobain dates for {0:,.0f} people in {1:,.1f} seconds!'.format(len(df), e-s))


In [ ]:
print('Getting from backup...')
df = df_bk




df[
    ['thename', 'birthdate', 'deathdate', 'cobaindate', 'birth_text', 'death_text']
][(df['cobaindate'] >= '2021-05-01') & (df['cobaindate'] <= '2021-05-31')].sort_values(by='cobaindate')


In [ ]:
# The article_lengths array (also Nx1 numpy) will hold length of each Wikip article in chars
article_lengths = np.zeros([len(deadpeople),1],dtype=np.int) 

for i in range(0,len(deadpeople)): # iterate through each name
#    print(i)
    # Construct the Wikip URL and read its contents
#    name = drf8str(deadpeople[i,0])
#    print(name)
#    name2 = urllib.parse.quote(name.replace(" ","_"))
#    print(name2)
#    url = "http://en.wikipedia.org/wiki/"+name2
    url = deadpeople[i,1]
#    print(i,":",url)
    page = urllib.request.urlopen(url).read()
    # Parse HTML to find the contents of every <p>...</p> tag
    soup = bs(page, "html.parser")
#    soup.prettify()
#    heading1 = soup.find('h1')
    x = soup.find_all('p')
    
    # Calculate the length of all the text within the <p>...</p> tags
    lent = 0
    for j in x:
        lent = lent + len(j.get_text())
    article_lengths[[i][0]] = lent
#    if  (i > 500):
#        print(i,":",url)
#        print(article_lengths[i][0])

    # instant gratification: print the loop variable every 50 turns so I see it's doing something
    if (i % 50 == 0):
        print(i)
    #   print(article_lengths[i][0])

# Add article length to list of names
deadpeople = np.hstack((deadpeople,article_lengths))

print('Done!')
print(deadpeople)

In [ ]:
# Results will eventually be in "deadpeople" - come back to it at the end

# Store the number of revisions & editors in a new array called "revisionarray"
revisionarray = np.zeros([len(deadpeople),2],dtype=int)

#revisionarray = np.zeros([1,2],dtype=int)
#for i in range(0,1):

# Iterate through all the names in deadpeople
for i in range(0,len(deadpeople)):
    name = str(deadpeople[i,3])
#    name = "David Bowie"
#    name = "Agha Nasir"
#    name = "Marvin Minsky"
#    name2 = urllib.parse.quote(name.replace(" ","_"))
    print(name)
    
    # Wikipedia displays revisions max 500 at a time. 
    iteration = 0
    completed = 0
    earliestrev = ''
    rownumber = 0
    
    while (completed == 0):
        iteration = iteration + 1

        mainurl = deadpeople[i,1].split('/')
        historyurl = ''
        for k in range(0,3):
            historyurl += str(mainurl[k])+'/'
        historyurl += 'w/index.php?title='
        historyurl += str(mainurl[4])        
        historyurl += "&offset="
        historyurl += earliestrev
        historyurl += "&limit=500&action=history"
        
        if (deadpeople[i,3] == 'Mike Hart'):
            historyurl = 'https://en.wikipedia.org/w/index.php?title=Mike_Hart_(singer/songwriter)&action=history'
#        print(historyurl)
        page = urllib.request.urlopen(historyurl).read()
        soup = bs(page, "html.parser")

        histolist = soup.find('ul',id='pagehistory')
        revisions = histolist.find_all('li')
        
        if (iteration == 1):
            revisor = np.empty([len(revisions),4],dtype='object')
        else:
            moreroom = np.empty(([len(revisions),4]),dtype='object')
            revisor = np.vstack((revisor,moreroom))

        if (len(revisions) < 500):
            completed = 1
        for j in revisions:
            if j.find(class_='history-deleted'):
                revisor[rownumber] = [dateutil.parser.parse('9999-12-31 23:59'),'','','']
            else:
                revisor[rownumber,0] = dateutil.parser.parse(j.find(class_='mw-changeslist-date').get_text())
                revisor[rownumber,1] = j.find(class_='mw-userlink').get_text()
                byte_change_string = ''
                byte_change_string = j.find(class_='history-size').get_text()        
                byte_change_string = byte_change_string.replace('(','')
                byte_change_string = byte_change_string.replace(')','')
                if (byte_change_string.find('bytes') > -1):
                    byte_change_string = byte_change_string.replace(' bytes','')
                elif (byte_change_string.find('byte') > -1):
                    byte_change_string = byte_change_string.replace(' byte','')                    
                byte_change_string = byte_change_string.replace(',','')
                if (byte_change_string == 'empty'):
                    revisor[rownumber,2] = 0
                else:
                    revisor[rownumber,2] = int(byte_change_string)
                plusorminusy = ''
                plusorminusy = j.find(class_='mw-plusminus-pos')
                if (plusorminusy == None):
                    plusorminusy = j.find(class_='mw-plusminus-neg')
                if (plusorminusy == None):
                    plusorminusy = j.find(class_='mw-plusminus-null')
                character_change_string = plusorminusy.get_text()
                character_change_string = character_change_string.replace('(','')
                character_change_string = character_change_string.replace(')','')
                character_change_string = character_change_string.replace('+','')
                character_change_string = character_change_string.replace(',','')
                revisor[rownumber,2] = int(character_change_string)
            rownumber = rownumber + 1
        earliestrev =  datetime.datetime.strftime(np.min(revisor[:,0]),'%Y%m%d%H%M%S')
    revisionarray[i,0] = (revisor.shape[0])
    revisionarray[i,1] = np.unique(revisor[:,[1]]).size
    print(i," : ",revisionarray[i])
deadpeople = np.hstack((deadpeople,revisionarray))
#print(deadpeople)
print("Done!")

In [ ]:
print('top')
linkarray = np.zeros([len(deadpeople),1],dtype=int)
#linkarray = np.zeros((1,1),dtype=int)
for i in range(0,len(deadpeople)):
#    print(i,'.',deadpeople[i,3])
    inbound_link_url = ''
    completed = 0
    nextpagelink = ''
    iteration = 0
    rownumber = 0
    while (completed == 0):
        iteration = iteration + 1
        if (nextpagelink == ''):
            mainurl = str(deadpeople[i,1]).split('/')
            for k in range(0,3):
                inbound_link_url += str(mainurl[k])+'/'
            inbound_link_url += 'w/index.php?title=Special:WhatLinksHere/'
            inbound_link_url += str(mainurl[4])
            inbound_link_url += "&limit=500"
        else:
            inbound_link_url = nextpagelink
        if (deadpeople[i,3] == 'Mike Hart'):
            inbound_link_url = 'https://en.wikipedia.org/wiki/Special:WhatLinksHere/Mike_Hart_(singer/songwriter)'
        try:
            page = urllib.request.urlopen(inbound_link_url).read()
        except:
            print('failed to parse on i=',i)
            print(i,':',inbound_link_url)

        soup = bs(page, "html.parser")
        nextpage_element = soup.find('a',string="next 500")
        if (nextpage_element != None):
            nextpagelink = "https://en.wikipedia.org/" + nextpage_element.get('href')
        else:
            completed = 1

        listoflinks = soup.find('ul',id='mw-whatlinkshere-list')

        inlinks = listoflinks.find_all('li')
              
        if (iteration == 1):
            linktracker = np.empty([len(inlinks),1],dtype=object)    
#            print('linktracker initialized to length ',len(linktracker))
        else:
#            print("size of linktracker before = ",linktracker.shape)
            moreroom = np.empty([len(inlinks),1],dtype=object)
#            print("size of moreroom = ",moreroom.shape)
            linktracker = np.vstack((linktracker,moreroom))
#            print("size of linktracker = ",linktracker.shape)
        
        for j in inlinks:
#            print(name)
#            print(rownumber)
#            print(j)
            linktracker[rownumber,0] = (bs.find(j,'a').get('href'))
            rownumber = rownumber + 1
        
#        print(linktracker)
    linkarray[i,0] = (linktracker.shape[0])
    print(i,':',deadpeople[i,3],':',linkarray[i,0])
#    print(linkarray[i,0])

deadpeople = np.hstack((deadpeople,linkarray))
print(deadpeople)

In [ ]:
#deadpeople = deadpeople[:,1:]
#print(deadpeople.shape)

x = pandas.DataFrame(deadpeople,columns=['text','url','death_date','name','age','nationality','cause','description','votes','sortorder','article_length','versions','editors','inbound_links'])
#x.set_index('id')
x.index.name = 'id'
#x
outfile = str(os.getcwd())+'/output/death10x.csv'
x.to_csv(outfile,encoding="utf-8")

print('Done!')

In [ ]:
print('Deaths...')
print('\tparsing text string...')
all_deaths_df = all_deaths_df.assign(name_and_more = np.nan)
all_deaths_df.loc[
    (all_deaths_df['death_text'].apply(lambda x: re.search(namefinder, x) != None)), 'name_and_more'
] = all_deaths_df[(all_deaths_df['death_text'].apply(lambda x: re.search(namefinder, x) != None))]['death_text'].apply(lambda x: x[re.search(namefinder,x).end():].strip())

print('\tgetting birth year as a check for matching...')
all_deaths_df = all_deaths_df.assign(birth_string = np.nan)
all_deaths_df.loc[all_deaths_df['death_text'].apply(lambda x: re.search(birthfinder, str(x)) != None), 
                  'birth_string'] = all_deaths_df[all_deaths_df['death_text'].apply(lambda x: re.search(birthfinder, str(x)) != None)]['death_text'].apply(lambda x: x[re.search(birthfinder, str(x)).start():re.search(birthfinder, str(x)).end()])

all_deaths_df = all_deaths_df.assign(birth_year_check = pandas.to_numeric(-1, downcast='integer', errors='coerce'))
all_deaths_df.loc[all_deaths_df['birth_string'].notnull(), 'birth_year_check'] = pandas.to_numeric(
    all_deaths_df[all_deaths_df['birth_string'].notnull()]['birth_string'].apply(lambda x: x[re.search('\d+',x).start():re.search('\d+',x).end()]), 
    downcast='integer', errors='coerce')

print('\tgetting names...')
all_deaths_df = all_deaths_df.assign(birth_excised = np.nan)
all_deaths_df.loc[
    all_deaths_df['name_and_more'].apply(lambda x: re.search(deathfinder, str(x)) != None), 
    'birth_excised'] = all_deaths_df[
                (all_deaths_df['name_and_more'].apply(lambda x: re.search(deathfinder, str(x)) != None))
                        ]['name_and_more'].apply(lambda x: x[:re.search(deathfinder, str(x)).start()] + x[re.search(deathfinder, str(x)).end():])
all_deaths_df.loc[all_deaths_df['birth_excised'].isnull(), 'birth_excised'] = all_deaths_df['name_and_more']
all_deaths_df = all_deaths_df.assign(thename = all_deaths_df['birth_excised'].apply(lambda x: str(x).split(',')[0].strip()))



print('\tFixing weirdness with names for people who died together...')
all_deaths_df.loc[all_deaths_df['thename'] == 'nan', 'thename'] = all_deaths_df[all_deaths_df['thename'] == 'nan']['death_text'].apply(lambda x: re.split(',',x)[0])
all_deaths_df = all_deaths_df[~(all_deaths_df['thename'].apply(lambda x: re.search('crew', x) != None))]
all_deaths_df = all_deaths_df[~(all_deaths_df['death_text'].apply(lambda x: re.search('Casualties', x) != None))]
all_deaths_df.loc[(all_deaths_df['death_text'].apply(lambda x: re.search('^\s*(–|-)\s*', x) != None)), 'thename'] = all_deaths_df[(all_deaths_df['death_text'].apply(lambda x: re.search('^\s*(–|-)\s*', x) != None))][ 'thename'].apply(lambda x: x[re.search('^\s*(–|-)\s*', str(x)).end():])
all_deaths_df = all_deaths_df.loc[~(all_deaths_df['death_text'].apply(lambda x: 'attack' in x.lower()))]                              
all_deaths_df = all_deaths_df[~all_deaths_df['death_text'].apply(lambda x: 'incident' in x.lower())]

print('\tRemoving duplicates...')
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'David Allen') & (all_deaths_df['themonth'] == 5) & (all_deaths_df['theday'] == 25))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Francisco Franco') & (all_deaths_df['themonth'] == 11) & (all_deaths_df['theday'] == 19))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Dirk Jan de Geer') & (all_deaths_df['themonth'] == 11) & (all_deaths_df['theday'] == 27))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Jyoti Basu') & (all_deaths_df['themonth'] == 1) & (all_deaths_df['theday'] == 16))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'John Marin') & (all_deaths_df['themonth'] == 10) & (all_deaths_df['theday'] == 1))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Bret Harte') & (all_deaths_df['themonth'] == 5) & (all_deaths_df['theday'] == 6))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'George Bouzianis') & (all_deaths_df['themonth'] == 10) & (all_deaths_df['theday'] == 22))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Udo Lattek') & (all_deaths_df['themonth'] == 2) & (all_deaths_df['theday'] == 1))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Jun Seba') & (all_deaths_df['themonth'] == 2) & (all_deaths_df['theday'] == 10))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'George Thorn') & (all_deaths_df['themonth'] == 1) & (all_deaths_df['theday'] == 13))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'George Hees') & (all_deaths_df['themonth'] == 6) & (all_deaths_df['theday'] == 10))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Edward Greenspan') & (all_deaths_df['themonth'] == 12) & (all_deaths_df['theday'] == 23))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Abdul Haris Nasution') & (all_deaths_df['themonth'] == 9) & (all_deaths_df['theday'] == 5))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Alfred de Grazia') & (all_deaths_df['themonth'] == 7) & (all_deaths_df['theday'] == 12))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Edmonia Lewis') & (all_deaths_df['death_year'] == 1911) )]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Laura Ingalls Wilder') & (all_deaths_df['themonth'] == 1) & (all_deaths_df['theday'] == 10))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Jean Moréas') & (all_deaths_df['themonth'] == 4) & (all_deaths_df['theday'] == 30))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Max Linder') & (all_deaths_df['themonth'] == 10) & (all_deaths_df['theday'] == 31))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Flora MacNeil') & (all_deaths_df['themonth'] == 5) & (all_deaths_df['theday'] == 16))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Tony Hancock') & (all_deaths_df['themonth'] == 6) & (all_deaths_df['theday'] == 24))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Charlie Byrd') & (all_deaths_df['themonth'] == 11) & (all_deaths_df['theday'] == 30))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Gene Raymond') & (all_deaths_df['themonth'] == 5) & (all_deaths_df['theday'] == 2))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Robert Mensah') & (all_deaths_df['themonth'] == 9) & (all_deaths_df['theday'] == 2))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'M. Visvesvaraya') & (all_deaths_df['themonth'] == 1) & (all_deaths_df['theday'] == 14))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Tito Puente') & (all_deaths_df['themonth'] == 5) & (all_deaths_df['theday'] == 31))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Bill Cowsill') & (all_deaths_df['themonth'] == 2) & (all_deaths_df['theday'] == 17))]
all_deaths_df = all_deaths_df[~((all_deaths_df['thename'] == 'Rebecca Clarke') & (all_deaths_df['death_year'] == 1981) )]

print('\tfixing people who died on February 29th...')
all_deaths_df = all_deaths_df[~all_deaths_df['thename'].apply(lambda x: re.search('^\d+\n+',x) != None)]#.shape#[['thename', 'death_text']]

print('\tfixing senior/junior...')
# all_deaths_df.loc[(all_deaths_df['death_text'] == 'Arthur M. Schlesinger') & (all_deaths_df['death_year'] == 1965), 'thename'] = 'Arthur M. Schlesinger, Sr.'
# all_deaths_df.loc[(all_deaths_df['death_text'] == 'Arthur M. Schlesinger') & (all_deaths_df['death_year'] == 2007), 'thename'] = 'Arthur M. Schlesinger, Jr.'
# all_deaths_df.loc[(all_deaths_df['death_text'] == 'Barack Obama') & (all_deaths_df['death_year'] == 1982), 'thename'] = 'Barack Obama, Sr.'
# all_deaths_df.loc[(all_deaths_df['death_text'] == 'John Paul Getty') & (all_deaths_df['death_year'] == 2003), 'thename'] = 'John Paul Getty, Jr.'
# all_deaths_df.loc[(all_deaths_df['death_text'] == 'Cal Ripken') & (all_deaths_df['death_year'] == 1999), 'thename'] = 'Cal Ripken, Sr.'




print('matching names...')
all_deaths_df = all_deaths_df[all_deaths_df['death_text'] != '1981']
all_births_df.loc[all_births_df['death_text'] == '1901 – Nina Bari, Russian mathematician (d. 1961)[11]', 'thename'] = 'Nina Bari'
all_deaths_df.loc[all_deaths_df['death_text'] == '1961.  – Nina Bari, Russian mathematician (b. 1901)[25]', 'thename'] = 'Nina Bari'
all_births_df.loc[all_births_df['death_text'] == '1915 – Eugène Martin, French racing driver (d. 2006)', 'thename'] = 'Eugène Martin'
all_deaths_df.loc[all_deaths_df['death_text'] == '2006.  – Eugène Martin, French race car driver (b. 1915)', 'thename'] = 'Eugène Martin'
all_births_df.loc[all_births_df['death_text'] == '1876 – 13th Dalai Lama (d. 1933)', 'thename'] = '13th Dalai Lama'




print('removing reference marks from names...')
all_births_df.loc[all_births_df['thename'].apply(lambda x: re.search('\[\d+\]',str(x)) != None)
                  , 'thename'] = all_births_df[
    all_births_df['thename'].apply(lambda x: re.search('\[\d+\]',str(x)) != None)
]['thename'].apply(lambda x: x[:re.search('\[\d+\]',str(x)).start()])

all_deaths_df.loc[
    all_deaths_df['thename'].apply(lambda x: re.search('\[\d+\]',str(x)) != None), 
    'thename'] = all_deaths_df[
    all_deaths_df['thename'].apply(lambda x: re.search('\[\d+\]',str(x)) != None)
]['thename'].apply(lambda x: x[:re.search('\[\d+\]',str(x)).start()])



print('backing up...')
# all_births_df_bk = all_births_df
# all_deaths_df_bk = all_deaths_df

e = time.time()
g = g + (e-s)
print('Found {0:,.0f} births and {1:,.0f} deaths in {2:,.0f} seconds!'.format(len(all_births_df), len(all_deaths_df), e-s))
#all_births_df[all_births_df['death_text'].apply(lambda x: 'laura bassi' in x.lower())]

#all_births_df[all_births_df['death_year_check'] != -1].groupby(['thename', 'death_year_check']).size().sort_values(ascending=False)

